## Generation of VDI Rasters
This notebook generates both a categorical and a continuous VDI raster for a given year, based on input rasters of two spectral indices and a linear model combining these indices.

In [12]:
import json
from osgeo import gdal
import math
import numpy as np
import sys

In this example, the VDI model combines MSAVI and TGSI. The model provides the slope of the linear model and the boundaries of the vegetation classes.

In [20]:
vdi_model_file = './models/vdi-msavi-tgsi.json'
x_index = 'msavi'
y_index = 'tgsi'
nodata_value = 255

with open(vdi_model_file, 'r') as json_file:
    model = json.load(json_file)

slope = model['slope']
breaks = model['classes-limits']
limits_vals = {x_index: model['x-axis-limits'], y_index: model['y-axis-limits']}

We open the MSAVI and TGSI raster files of the year 2022 and read them as numpy arrays.

In [21]:
# Open the GeoTIFF files
xfile_name = './rasters/msavi_2022.tif'
yfile_name = './rasters/tgsi_2022.tif'
output_cont_file  = './rasters/vdi_cont_2022.tif'
output_cat_file  = './rasters/vdi_cat_2022.tif'

xfile = gdal.Open(xfile_name)
xarray = xfile.GetRasterBand(1).ReadAsArray()
yfile = gdal.Open(yfile_name)
yarray= yfile.GetRasterBand(1).ReadAsArray()

In [23]:
# Get the number of columns and rows (i.e., the image dimensions)
xsize = xfile.RasterXSize
ysize = yfile.RasterYSize
print(f'Raster dimensions: {xsize}*{ysize} (total number of pixels: {xsize * ysize})')

Raster dimensions: 38066*20850 (total number of pixels: 793676100)


This function normalizes the values of each array of spectral indices.

In [ ]:
def normalize(ind,axis):
    return (ind - limits_vals[axis][0]) / (limits_vals[axis][1] - limits_vals[axis][0])

- Calculate the resulting VDI array as continuous values.
- Calculate the resulting VDI array as discrete (categorical) values.

In [16]:
cont_raster = np.zeros((xarray.shape[0],xarray.shape[1]),dtype=np.float32)
cat_raster = np.zeros((xarray.shape[0],xarray.shape[1]),dtype=np.uint8)
for i in range(xarray.shape[0]):
    for j in range(xarray.shape[1]):
        if math.isfinite(xarray[i,j]):
            cont_raster[i,j] = slope * normalize(xarray[i,j],x_index)-normalize(yarray[i,j],y_index)
            if cont_raster[i,j] <= breaks[1]:
                cat_raster[i,j] = 0
            elif cont_raster[i,j] <= breaks[2]:
                cat_raster[i,j] = 1
            elif cont_raster[i,j] <= breaks[3]:
                cat_raster[i,j] = 2
            elif cont_raster[i,j] <= breaks[4]:
                cat_raster[i,j] = 3
            else:
                cat_raster[i,j] = 4
        else:
            cont_raster[i,j] = nodata_value 
            cat_raster[i,j] = nodata_value

Save the continuous VDI raster in geotiff format.

In [17]:
# Create a new TIFF file for the modified image
driver = gdal.GetDriverByName("GTiff")
out_cont = driver.Create(output_cont_file, xsize, ysize, 1, gdal.GDT_Float32)

# Write the modified image to the output file
out_band = out_cont.GetRasterBand(1)
out_band.WriteArray(cont_raster)
out_band.SetNoDataValue(nodata_value)

# Set the georeferencing information from the input file
out_cont.SetGeoTransform(xfile.GetGeoTransform())
out_cont.SetProjection(xfile.GetProjection())

Warning 1: PROJ: proj_create_from_database: Open of /home/jovyan/.conda-envs/myenv/share/proj failed


0

Save the categorical VDI raster in geotiff format.

In [18]:
# Create a new TIFF file for the modified image
driver = gdal.GetDriverByName("GTiff")
out_cat = driver.Create(output_cat_file, xsize, ysize, 1, gdal.GDT_Byte)

# Write the modified image to the output file
out_band = out_cat.GetRasterBand(1)
out_band.WriteArray(cat_raster)
out_band.SetNoDataValue(nodata_value)

# Set the georeferencing information from the input file
out_cat.SetGeoTransform(xfile.GetGeoTransform())
out_cat.SetProjection(xfile.GetProjection())

# Close the input and output files
xfile = None
yfile = None
out_cont = None 
out_cat = None 


ERROR 1: PROJ: proj_create_from_name: Open of /home/jovyan/.conda-envs/myenv/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /home/jovyan/.conda-envs/myenv/share/proj failed
